## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centres in Helsinki**.

Problem 1 concerns the locations of shopping centres: find their addresses and translate them into coordinates.

---

### a) Prepare an input file containing the addresses of shopping centres

Find out the addresses of the following shopping centres (e.g., by using your favourite search engine), and collect them in a text file called `shopping_centres.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
The text file should be in semicolon-separated format (`;`) and include the following columns:

- `id` (integer) a unique identifier for each shopping centre (a
- `name` (string) of each shopping center
- `addr` (string) the address 


See an example of how to format the text file [in the lesson 3 materials](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/geocoding-in-geopandas.html). Remember to *add*, *commit*, and *push* the file to your git repository.

---


### b) Read the list of addresses

Read the list of addresses you just prepared into a `pandas.DataFrame` called `shopping_centres`

In [1]:
import pathlib
import json
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

/tmp/ipykernel_4679/2200372433.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
with open("query_7.json", "r") as read_file:
    json_data = json.load(read_file)
json_to_df = json_data["features"]

In [3]:
# ADD YOUR OWN CODE HERE
my_list = []
for feature in json_to_df:
    my_list.append(
        {'id': feature["id"],
         'name': feature["properties"]["INF_NOME"],
         'addr': f'{feature["properties"]["INF_MORADA"]}'
        }        
    )
shopping_centres = pd.DataFrame(my_list)
shopping_centres

,id,name,addr
0,1,Itis,"Itäkatu 1-7, 00930 Helsinki"
1,2,Forum,"Mannerheimintie 20, 00100 Helsinki"
2,3,Iso-omena,"Piispansilta 11, 02230 Espoo"
3,4,Sello,"Leppävaarankatu 3-9, 02600 Espoo"
4,5,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa"
5,6,REDI,"Hermannin rantatie 5, 00580 Helsinki"
6,7,Tripla,"Fredikanterassi 1, 00520 Helsinki"


In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import pandas
assert isinstance(shopping_centres, pandas.DataFrame)
for column in ("id", "name", "addr"):
    assert column in shopping_centres.columns


---

### c) Geocode the addresses

Geocode the addresses using the Nominatim geocoding service. Join the results with the input data, and store them in a `geopandas.GeoDataFrame` with the same name (`shopping_centres`). 

Remember to define a custom `user_agent` string!

In [5]:
# ADD YOUR OWN CODE HERE
geocoded_addresses = geocode(
    shopping_centres["addr"],
    provider="nominatim",
    user_agent="autogis2022",
    timeout=10
)
geocoded_addresses.head()

,geometry,address
0,POINT (25.08262 60.21191),"Burger King, 1-7, Itäkatu, Itäkeskus, Vartioky..."
1,POINT (24.93798 60.16954),"Momochi, 20, Mannerheimintie, Keskusta, Kluuvi..."
2,POINT (24.73795 60.16272),"11, Piispansilta, Matinkylä, Suur-Matinkylä, E..."
3,POINT (24.81346 60.21845),"Elisa, 3-9, Leppävaarankatu, Säteri, Etelä-Lep..."
4,POINT (24.96282 60.29245),"Stockmann, 3, Vantaanportinkatu, Vantaanportti..."


In [6]:
shopping_centres = geocoded_addresses[["geometry"]].join(shopping_centres)

In [7]:
shopping_centres

,geometry,id,name,addr
0,POINT (25.08262 60.21191),1,Itis,"Itäkatu 1-7, 00930 Helsinki"
1,POINT (24.93798 60.16954),2,Forum,"Mannerheimintie 20, 00100 Helsinki"
2,POINT (24.73795 60.16272),3,Iso-omena,"Piispansilta 11, 02230 Espoo"
3,POINT (24.81346 60.21845),4,Sello,"Leppävaarankatu 3-9, 02600 Espoo"
4,POINT (24.96282 60.29245),5,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa"
5,POINT (24.97819 60.18730),6,REDI,"Hermannin rantatie 5, 00580 Helsinki"
6,POINT (24.93051 60.19826),7,Tripla,"Fredikanterassi 1, 00520 Helsinki"


In [8]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
assert isinstance(shopping_centres, geopandas.GeoDataFrame)
for column in ("id", "name", "addr", "geometry"):
    assert column in shopping_centres.columns

Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [9]:
# ADD YOUR OWN CODE HERE
shopping_centres = shopping_centres.to_crs("EPSG:3879")

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import pyproj
assert shopping_centres.crs == pyproj.CRS("EPSG:3879")


---

### d) Save the resulting vector data set

Save `shopping_centres` as a *GeoPackage* named `shopping_centres.gpkg`:

In [11]:
# ADD YOUR OWN CODE HERE
shopping_centres.to_file('shopping_centres.gpkg', driver='GPKG')


---

Well done! Now you can continue to [problem 2](Exercise-3-Problem-2.ipynb)